Start jupyter-lab

```bash
jupyter-lab --notebook-dir=$HOME/gits/gerashegalov/rapids-shell/src/jupyter
```
or simply open in VS Code with Jupyter extension

# Repro for [NVIDIA/spark-rapids#8625](https://github.com/NVIDIA/spark-rapids/issues/8625)

In [ ]:
import datetime
import findspark
import os

In [ ]:
rapids_version = '23.08.0-SNAPSHOT'
spark_version = '3.4.1'

In [ ]:
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_LOCAL_IP'] = '127.0.0.1'
os.environ['TZ'] = 'UTC'

home_dir = os.environ['HOME']
work_dir = f"{home_dir}/jupyter_run_dir"
spark_home = f"{home_dir}/dist/spark-3.4.1-bin-hadoop3"
cores_per_exec = 1
spark_master = f"local[{cores_per_exec}]"

In [ ]:
findspark.init(spark_home = spark_home)
findspark.add_jars(f"{home_dir}/gits/NVIDIA/spark-rapids/dist/target/rapids-4-spark_2.12-{rapids_version}-cuda11.jar")
import pyspark
spark = pyspark.sql.SparkSession.builder \
    .appName('RAPIDS PySpark Notebook') \
    .master(spark_master) \
    .config(map = {
        'spark.plugins': 'com.nvidia.spark.SQLPlugin',
        'spark.rapids.sql.explain': 'ALL', 
        'spark.sql.adaptive.enabled': False,
        'spark.driver.extraJavaOptions': '-Dai.rapids.cudf.preserve-dependencies=true',
    }) \
    .getOrCreate()

In [ ]:
ts = datetime.datetime(5000, 7, 14, 7, 38, 45, 418688)
ts

In [ ]:
spark.createDataFrame([(ts,),], 'ts_ntz timestamp').createOrReplaceTempView('df1')
spark.sql("SELECT * FROM df1").show(truncate = False)
spark.sql("SELECT * FROM df1").printSchema()

In [ ]:
spark.conf.get('spark.sql.parquet.outputTimestampType')

In [ ]:
spark.sql("SELECT MAX(*) FROM df1").write.mode('overwrite').parquet('/tmp/int96_out.parquet')